# Project 8: ML for Business
---
You work for the OilyGiant mining company. Your task is to find the best place for a new well.
You have data on oil samples from three regions. Parameters of each oil well in the region are already known. Build a model that will help to pick the region with the highest profit margin. Analyze potential profit and risks using the Bootstrapping technique.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, recall_score, accuracy_score, f1_score, precision_score
from sklearn.linear_model import LinearRegression
from numpy.random import RandomState

## 1) Dowload and Prepare Data
---

In [2]:
g0 = pd.read_csv('geo_data_0.csv')
g1 = pd.read_csv('geo_data_1.csv')
g2 = pd.read_csv('geo_data_2.csv')

### Region 1:
---

In [3]:
g0.head(3)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


In [4]:
g0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
g0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


### Region 2:
---

In [6]:
g1.head(3)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


In [7]:
g1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
g1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


### Region 3:
---

In [9]:
g2.head(3)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


In [10]:
g2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
g2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


### The Data:
- 100k Datapoints for each location with no null values in any column
- Each set includes a unique id for the oil well, 3 feature variables, and a volume of reserves
- Region 3 (95) has the highest average reserve volume followed closely by Region 1 (92.5) and then Region 2 (68.825)
- They are ranked the same at the 50% (median) and all have a positive skew, although Region 2's skew is much stornger than that of the other two
- Each region also has a similar stdev on oil reserves around 44-45

## 2) Train and Test the Model for Each Region
---

2.1. Split the data into a training set and validation set at a ratio of 75:25.

In [12]:
# Drop id and use f#s as features and product as target variable

# Region 1
feat0 = g0.drop(['id', 'product'], axis=1)
targ0 = g0['product']
x0_train, x0_val, y0_train, y0_val = train_test_split(feat0, targ0, test_size=0.25)

# Region 2
feat1 = g1.drop(['id', 'product'], axis=1)
targ1 = g1['product']
x1_train, x1_val, y1_train, y1_val = train_test_split(feat1, targ1, test_size=0.25)

# Region 3
feat2 = g2.drop(['id', 'product'], axis=1)
targ2 = g2['product']
x2_train, x2_val, y2_train, y2_val = train_test_split(feat2, targ2, test_size=0.25)

2.2. Train the model and make predictions for the validation set.

In [13]:
# Region 1
m0 = LinearRegression()
m0.fit(x0_train, y0_train)
pred0 = m0.predict(x0_val)

In [14]:
# Region 2
m1 = LinearRegression()
m1.fit(x1_train, y1_train)
pred1 = m1.predict(x1_val)

In [15]:
# Region 3
m2 = LinearRegression()
m2.fit(x2_train, y2_train)
pred2 = m2.predict(x2_val)

2.3. Save the predictions and correct answers for the validation set.

In [16]:
pt0 = pd.DataFrame({'actual':y0_val, 'predicted':pred0})
display(pt0.head(3))
mse0 = mean_squared_error(y0_val, pred0)
print('Region 1 MSE:', mse0)

,actual,predicted
99484,85.497486,105.372543
60699,122.483443,101.661233
10519,157.437601,80.249649


Region 1 MSE: 1414.878166701268


In [17]:
pt1 = pd.DataFrame({'actual':y1_val, 'predicted':pred1})
display(pt1.head(3))
mse1 = mean_squared_error(y1_val, pred1)
print('Region 2 MSE:', mse1)

,actual,predicted
6360,84.038886,82.872090
23085,134.766305,134.083341
96002,53.906522,54.092360


Region 2 MSE: 0.787604237972845


In [18]:
pt1 = pd.DataFrame({'actual':y1_val, 'predicted':pred1})
display(pt1.head(3))
mse2 = mean_squared_error(y2_val, pred2)
print('Region 3 MSE:', mse2)

,actual,predicted
6360,84.038886,82.872090
23085,134.766305,134.083341
96002,53.906522,54.092360


Region 3 MSE: 1585.6261340771034


2.4. Print the average volume of predicted reserves and model RMSE.

In [19]:
print('Region 1: \n Average predicted volume: {:.2f} | Model RMSE: {:.2f}'.format(pred0.mean(), np.sqrt(mse0)))
print()
print('Region 2: \n Average predicted volume: {:.2f} | Model RMSE: {:.2f}'.format(pred1.mean(), np.sqrt(mse1)))
print()
print('Region 3: \n Average predicted volume: {:.2f} | Model RMSE: {:.2f}'.format(pred2.mean(), np.sqrt(mse2)))

Region 1: 
 Average predicted volume: 92.71 | Model RMSE: 37.61

Region 2: 
 Average predicted volume: 68.64 | Model RMSE: 0.89

Region 3: 
 Average predicted volume: 94.82 | Model RMSE: 39.82


2.5. Analyze the results.

- Using a simple LinearRegression model for each region I was able to build a model fro each region and make prediction on the 25% validation data.
- The model using the region 2 validation data had the lowest RMSE by far meaning its predictions were the closest to the true values.
- Although it had the highest RMSE, it also had the lowest predicted oil volume compared to the other two regions.
- Regions 1 and 3 were both very similar in volume and RMSE.

## 3) Profit Calculation
---

3.1. Store all key values for calculations in separate variables.

In [20]:
two_hundred_well_budget = 100000000 # 100 million USD
one_well_budget = 500000 # 500k USD
barrel_revenue = 4.5 #USD
one_unit_product_revenue = 4500 #USD
risk_threshold = 0.025 # percent %

3.2. Calculate the volume of reserves sufficient for developing a new well without losses. Compare the obtained value with the average volume of reserves in each region.

In [21]:
# volume * unit revenue = one well budget
# volume = one well budget / unit revenue

volume_breakeven = one_well_budget / one_unit_product_revenue
print('Break-even point for volume revenue to exceed new well budget:\n --> {:.2f}'.format(volume_breakeven))
print()
print('Region 1 average predicted volume: {:.2f}'.format(pred0.mean()))
print('Region 2 average predicted volume: {:.2f}'.format(pred1.mean()))
print('Region 3 average predicted volume: {:.2f}'.format(pred2.mean()))

Break-even point for volume revenue to exceed new well budget:
 --> 111.11

Region 1 average predicted volume: 92.71
Region 2 average predicted volume: 68.64
Region 3 average predicted volume: 94.82


3.3. Provide the findings about the preparation for profit calculation step.

- No region has an average predicted volume that is greated than the breakeven revenue for building a new well.
- Region 1 is ~19 units below
- Region 2 is ~43 units below
- Region 3 is ~16 units below

## 4) Write a function to calculate profit from a set of selected oil wells and model predictions:
---

4.1. Pick the wells with the highest values of predictions.

In [22]:
def profit_calc(df, predictions):
    
    '''
    This function. takes in one of the regional dataframes and returns a dataframe
    of the top 200 wells by predicted product amount as well as the total profit calculated.
    '''
    
    df = df.copy()
    df['pred_product'] = predictions
    df_top = df.sort_values('pred_product', ascending=False).head(200)
    
    df_top['revenue'] = one_unit_product_revenue * df_top['pred_product']
    df_top['profit'] = df_top['revenue'] - one_well_budget
    
    tot_profit = sum(df_top['profit'])
    
    return df_top, tot_profit

In [23]:
r1_top, r1_profit = profit_calc(x0_val, pred0)

print('Sum of Profit: ${:.2f}'.format(r1_profit))

Sum of Profit: $39719127.57


In [24]:
r2_top, r2_profit = profit_calc(x1_val, pred1)

print('Sum of Profit: ${:.2f}'.format(r2_profit))

Sum of Profit: $24866617.53


In [25]:
r3_top, r3_profit = profit_calc(x2_val, pred2)

print('Sum of Profit: ${:.2f}'.format(r3_profit))

Sum of Profit: $33297827.13


4.2. Summarize the target volume of reserves in accordance with these predictions

- Looking into the profit for the top 200 wells by predicted profit in each region:
    - Region 1 brings in the most profit
        - If you assume worst case scenario on the lowest prediction, we can subtract the RMSE (37) from the predicted product amount (148) we get 111 which is almost exactly the breakeven point
        - Here we see that even with a higher RMSE, we can expect maybe only a couple of the top 200 wells in region 1 to not break even
    - Region 3 is not far behind, but it also has a higher RMSE, increasing potential negative variability
    - Region 2 has the lowest total profit, but the lowest RMSE by far

4.3. Provide findings: suggest a region for oil wells' development and justify the choice. Calculate the profit for the obtained volume of reserves.

- I would suggest Region 1 for developing oil wells.
- Based on the models predicted volumes, RMSE, projected revenue/profit, and budget we can expect just about every well to break even.
- The top wells can be expected to make upwards of 250,000 USD each

## 5) Calculate risks and profit for each region:
---

5.1. Use the bootstrapping technique with 1000 samples to find the distribution of profit.

In [26]:
def region_boot(series):
    data = series
    state = np.random.RandomState(123)
    
    distr = []
    for i in range(1000):
        sub = data.sample(n=1, replace=True, random_state=state)
        distr.append(sub.values)

    distr = pd.Series(distr)
    lower = distr.quantile(q=0.025)
    upper = distr.quantile(q=0.975)
    mean = distr.mean()
    return mean, lower, upper

In [27]:
m1, l1, u1 = region_boot(r1_top['profit'])
m2, l2, u2 = region_boot(r2_top['profit'])
m3, l3, u3 = region_boot(r3_top['profit'])

print('Region 1: Average Profit:', m1)
print('          2.5% Quantile:', l1)
print('          97.5% Quantile:', u1)
print()
print('Region 2: Average Profit:', m2)
print('          2.5% Quantile:', l2)
print('          97.5% Quantile:', u2)
print()
print('Region 3: Average Profit:', m3)
print('          2.5% Quantile:', l3)
print('          97.5% Quantile:', u3)



Region 1: Average Profit: [199371.05227475]
          2.5% Quantile: [168278.84192537]
          97.5% Quantile: [280217.50471895]

Region 2: Average Profit: [124366.88469351]
          2.5% Quantile: [122987.041618]
          97.5% Quantile: [128032.04903126]

Region 3: Average Profit: [167212.22953954]
          2.5% Quantile: [141597.67076637]
          97.5% Quantile: [242656.14973784]


- Again this data shows that Region 1 should be the region to develop in.
- Here we have the highest average profit as well as the highest bounds within the 95% confidence interval